# Initialisation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
pip install python-binance

Defaulting to user installation because normal site-packages is not writeable
  Using cached python_binance-1.0.16-py2.py3-none-any.whl (65 kB)
  Using cached dateparser-1.1.1-py2.py3-none-any.whl (288 kB)
  Using cached ujson-5.4.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (45 kB)
  Using cached websockets-10.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (111 kB)
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
  Using cached pytz_deprecation_shim-0.1.0.post0-py2.py3-none-any.whl (15 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 656.7 kB/s eta 0:00:000:00:01m eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install ta

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29103 sha256=f00a93b0bb337e195cace669c360d2e576177b658cb51084f377e961d8a73a3b
  Stored in directory: /home/mahdi/.cache/pip/wheels/1b/81/50/db21d9643326c61dd6fee1dc2562d8351ee7753164c24ead38
Successfully built ta
Note: you may need to restart the kernel to use updated packages.


## Initialisation paramettres

In [5]:
# -- Import --
import pandas as pd
import numpy as np
from binance.client import Client
import ta
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import time
from math import *
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [6]:
#initialisation des cryptos 
cryptos= ['ETHUSDT','BTCUSDT']

# initialisation des dates 
startDate = "2022-01-15"
endDate = "2022-02-26" # une date ou  ""
timeInterval = '1h'

# Plage des paramettres à tester 
stochTop_plage = np.arange(80,95,2)/100
stochBottom_plage = np.arange(10,25,2)/100
stoch_rsi_plage = np.arange(12,16,1)
trixLength_plage = np.arange(9,12,1)
trixSignal_plage = np.arange(21,25,1)
ema200 =200

## Load data

In [7]:
def load_data(list_cryptos, timeInterval, startDate, endDate, Client):
  # -- Define Binance Client --
  client = Client()
  dict_cryptos = {}
  for pairName in list_cryptos :
    # -- Load all price data from binance API --
    klinesT = client.get_historical_klines(pairName, timeInterval, startDate, endDate)

    # -- Define your dataset --
    dict_cryptos[pairName] = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
                                        'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    dict_cryptos[pairName]['close'] = pd.to_numeric(dict_cryptos[pairName]['close'])
    dict_cryptos[pairName]['high'] = pd.to_numeric(dict_cryptos[pairName]['high'])
    dict_cryptos[pairName]['low'] = pd.to_numeric(dict_cryptos[pairName]['low'])
    dict_cryptos[pairName]['open'] = pd.to_numeric(dict_cryptos[pairName]['open'])

    # -- Set the date to index --
    dict_cryptos[pairName] = dict_cryptos[pairName].set_index(dict_cryptos[pairName]['timestamp'])
    dict_cryptos[pairName].index = pd.to_datetime(dict_cryptos[pairName].index, unit='ms')
    del dict_cryptos[pairName]['timestamp']

    #print(f"Data loaded 100% of {pairName} shape : {dict_cryptos[pairName].shape} ")    
  return dict_cryptos

## Define indicator

In [8]:
def define_indicator(dict_cryptos, list_cryptos, trixLength, trixSignal, ema200, stoch_rsi ) :    
  for pairName in list_cryptos:
    # -- Drop all columns we do not need --
    dict_cryptos[pairName].drop(dict_cryptos[pairName].columns.difference(['open','high','low','close','volume']), 1, inplace=True)

    # -- Indicators, you can edit every value --
    dict_cryptos[pairName]['EMA200'] = ta.trend.ema_indicator(close=dict_cryptos[pairName]['close'], window=ema200)

    # -- Trix Indicator --
    dict_cryptos[pairName]['TRIX'] = ta.trend.ema_indicator(ta.trend.ema_indicator(ta.trend.ema_indicator(close=dict_cryptos[pairName]['close'], window=trixLength), window=trixLength), window=trixLength)
    dict_cryptos[pairName]['TRIX_PCT'] = dict_cryptos[pairName]["TRIX"].pct_change()*100
    dict_cryptos[pairName]['TRIX_SIGNAL'] = ta.trend.sma_indicator(dict_cryptos[pairName]['TRIX_PCT'],trixSignal)
    dict_cryptos[pairName]['TRIX_HISTO'] = dict_cryptos[pairName]['TRIX_PCT'] - dict_cryptos[pairName]['TRIX_SIGNAL']

    # -- Stochasitc RSI --
    dict_cryptos[pairName]['STOCH_RSI'] = ta.momentum.stochrsi(close=dict_cryptos[pairName]['close'], window=stoch_rsi, smooth1=3, smooth2=3)
    print(f"Indicators loaded 100% for {pairName}")
    # -- Uncomment the line below if you want to check your dataset with indicators --
  return dict_cryptos

## Traiding V2

In [25]:
from time import perf_counter
from threading import Thread

start_time = perf_counter()


# def algo_traiding(dict_df_indicated, result_data_frame, startDate, endDate, stochTop, stochBottom):
dict_cryptos = load_data(cryptos, timeInterval, startDate, endDate, Client)
result_data_frame = pd.DataFrame(columns=["pairName",
                                          "trixLength",
                                          "trixSignal",
                                          "ema200",
                                          "stochTop",
                                          "stochBottom",
                                          "stoch_rsi",
                                          "Period start",
                                          "Period end",
                                          "initalWallet",
                                          "Final balance ($)",
                                          "Performance vs US Dollar (%)",
                                          "Buy and Hold Performence(%) ",
                                          "Performance vs Buy and Hold (%)",
                                          "Best trade (%)",
                                          "Worst trade (%)",
                                          "Worst drawBack(%)",
                                          "Total fees ($)",
                                          "Total trades on period",
                                          "Number of positive trades",
                                          "Number of negative trades",
                                          "Trades win rate ratio (%)",
                                          "Average trades performance (%)",
                                          "Average positive trades (%)",
                                          "Average negative trades (%)"])


# -- Condition to BUY market --
def buyCondition(row, previousRow):
    if row['TRIX_HISTO'] > 0 and row['STOCH_RSI'] < stochTop:
        return True
    else:
        return False

# -- Condition to SELL market --
def sellCondition(row, previousRow):
    if row['TRIX_HISTO'] < 0 and row['STOCH_RSI'] > stochBottom:
        return True
    else:
        return False

i = 0
    
def trainding_v2():
    global i, endDate, result_data_frame
    i = i + 1
    print(i, '\n')
    print(trixLength, trixSignal, stochTop, stochBottom, stoch_rsi)

    dict_df_indicated = define_indicator(dict_cryptos, cryptos, trixLength, trixSignal, ema200, stoch_rsi)
    # result_data_frame = result_data_frame.append(algo_traiding(dict_df_indicated, result_data_frame, startDate, endDate, stochTop, stochBottom))

    dico_dt = {}
    for pairName in dict_df_indicated:
        dfTest = None
        # dfTest = df.copy()

        if endDate == "":
            endDate = None

        # -- Ifyou want to run your BackTest on a specific period, uncomment the line below --
        dfTest = dict_df_indicated[pairName][startDate: endDate]

        # -- Definition of dt, that will be the dataset to do your trades analyses --
        dt = None
        dt = pd.DataFrame(
            columns=['date', 'position', 'reason', 'price', 'frais', 'fiat', 'coins', 'wallet', 'drawBack'])

        # -- You can change variables below --
        usdt = 1000
        makerFee = 0.0002
        takerFee = 0.0007

        # -- Do not touch these values --
        initalWallet = usdt
        wallet = usdt
        coin = 0
        lastAth = 0
        previousRow = dfTest.iloc[0]
        stopLoss = 0
        takeProfit = 500000
        buyReady = True
        sellReady = True

        

        # -- Iteration on all your price dataset (df) --
        for index, row in dfTest.iterrows():
            # -- Buy market order --
            if buyCondition(row, previousRow) and usdt > 0 and buyReady == True:
                # -- You can define here at what price you buy --
                buyPrice = row['close']

                # -- Define the price of you SL and TP or comment it if you don't want a SL or TP --
                # stopLoss = buyPrice - 0.02 * buyPrice
                # takeProfit = buyPrice + 0.04 * buyPrice

                coin = usdt / buyPrice
                fee = takerFee * coin
                coin = coin - fee
                usdt = 0
                wallet = coin * row['close']

                # -- Check if your wallet hit a new ATH to know the drawBack --
                if wallet > lastAth:
                    lastAth = wallet

                # -- You can uncomment the line below if you want to see logs --
                # print("Buy COIN at",buyPrice,'$ the', index)

                # -- Add the trade to DT to analyse it later --
                myrow = {'date': index, 'position': "Buy", 'reason': 'Buy Market Order', 'price': buyPrice,
                         'frais': fee, 'fiat': usdt, 'coins': coin, 'wallet': wallet,
                         'drawBack': (wallet - lastAth) / lastAth}
                dt = dt.append(myrow, ignore_index=True)

            # -- Stop Loss --
            elif row['low'] < stopLoss and coin > 0:
                sellPrice = stopLoss
                usdt = coin * sellPrice
                fee = makerFee * usdt
                usdt = usdt - fee
                coin = 0
                buyReady = False
                wallet = usdt

                # -- Check if your wallet hit a new ATH to know the drawBack --
                if wallet > lastAth:
                    lastAth = wallet

                # -- You can uncomment the line below if you want to see logs --
                # print("Sell COIN at Stop Loss",sellPrice,'$ the', index)

                # -- Add the trade to DT to analyse it later --
                myrow = {'date': index, 'position': "Sell", 'reason': 'Sell Stop Loss', 'price': sellPrice,
                         'frais': fee, 'fiat': usdt, 'coins': coin, 'wallet': wallet,
                         'drawBack': (wallet - lastAth) / lastAth}
                dt = dt.append(myrow, ignore_index=True)

                # -- Sell Market Order --
            elif sellCondition(row, previousRow) and coin > 0 and sellReady == True:

                # -- You can define here at what price you buy --
                sellPrice = row['close']
                usdt = coin * sellPrice
                fee = takerFee * usdt
                usdt = usdt - fee
                coin = 0
                buyReady = True
                wallet = usdt

                # -- Check if your wallet hit a new ATH to know the drawBack --
                if wallet > lastAth:
                    lastAth = wallet

                # -- You can uncomment the line below if you want to see logs --
                # print("Sell COIN at",sellPrice,'$ the', index)

                # -- Add the trade to DT to analyse it later --
                myrow = {'date': index, 'position': "Sell", 'reason': 'Sell Market Order', 'price': sellPrice,
                         'frais': fee, 'fiat': usdt, 'coins': coin, 'wallet': wallet,
                         'drawBack': (wallet - lastAth) / lastAth}
                dt = dt.append(myrow, ignore_index=True)

            previousRow = row

        # -- BackTest Analyses --
        dt = dt.set_index(dt['date'])
        dt.index = pd.to_datetime(dt.index)
        dt['resultat'] = dt['wallet'].diff()
        dt['resultat%'] = dt['wallet'].pct_change() * 100
        dt.loc[dt['position'] == 'Buy', 'resultat'] = None
        dt.loc[dt['position'] == 'Buy', 'resultat%'] = None

        dt['tradeIs'] = ''
        dt.loc[dt['resultat'] > 0, 'tradeIs'] = 'Good'
        dt.loc[dt['resultat'] <= 0, 'tradeIs'] = 'Bad'

        iniClose = dfTest.iloc[0]['close']
        lastClose = dfTest.iloc[len(dfTest) - 1]['close']
        holdPercentage = ((lastClose - iniClose) / iniClose) * 100
        algoPercentage = ((wallet - initalWallet) / initalWallet) * 100
        vsHoldPercentage = ((algoPercentage - holdPercentage) / holdPercentage) * 100

        try:
            tradesPerformance = round(dt.loc[(dt['tradeIs'] == 'Good') | (dt['tradeIs'] == 'Bad'), 'resultat%'].sum()
                                      / dt.loc[
                                          (dt['tradeIs'] == 'Good') | (dt['tradeIs'] == 'Bad'), 'resultat%'].count(), 2)
        except:
            tradesPerformance = 0
            print("/!\ There is no Good or Bad Trades in your BackTest, maybe a problem...")

        try:
            totalGoodTrades = dt.groupby('tradeIs')['date'].nunique()['Good']
            AveragePercentagePositivTrades = round(dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].sum()
                                                   / dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].count(), 2)
            idbest = dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].idxmax()
            bestTrade = str(
                round(dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].max(), 2))
        except:
            totalGoodTrades = 0
            AveragePercentagePositivTrades = 0
            idbest = ''
            bestTrade = 0
            print("/!\ There is no Good Trades in your BackTest, maybe a problem...")

        try:
            totalBadTrades = dt.groupby('tradeIs')['date'].nunique()['Bad']
            AveragePercentageNegativTrades = round(dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].sum()
                                                   / dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].count(), 2)
            idworst = dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].idxmin()
            worstTrade = round(dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].min(), 2)
        except:
            totalBadTrades = 0
            AveragePercentageNegativTrades = 0
            idworst = ''
            worstTrade = 0
            print("/!\ There is no Bad Trades in your BackTest, maybe a problem...")

        totalTrades = totalBadTrades + totalGoodTrades
        # print(totalTrades)
        winRateRatio = (totalGoodTrades / totalTrades) * 100

        reasons = dt['reason'].unique()
        dico_dt[pairName] = dt
        """
        print("Pair Symbol :",pairName)
        print("Period : [" + str(dfTest.index[0]) + "] -> [" +
              str(dfTest.index[len(dfTest)-1]) + "]")
        print("Starting balance :", initalWallet, "$")

        print("\n----- General Informations -----")
        print("Final balance :", round(wallet, 2), "$")
        print("Performance vs US Dollar (%):", round(algoPercentage, 2), "%")
        print("Buy and Hold Performence :", round(holdPercentage, 2), "%")
        print("Performance vs Buy and Hold :", round(vsHoldPercentage, 2), "%")
        print("Best trade : ",bestTrade, "%, the", idbest)
        print("Worst trade :", worstTrade, "%, the", idworst)
        print("Worst drawBack :", str(100*round(dt['drawBack'].min(), 2)), "%")
        print("Total fees : ", round(dt['frais'].sum(), 2), "$")

        print("\n----- Trades Informations -----")
        print("Total trades on period :",totalTrades)
        print("Number of positive trades :", totalGoodTrades)
        print("Number of negative trades : ", totalBadTrades)
        print("Trades win rate ratio :", round(winRateRatio, 2), '%')
        print("Average trades performance :",tradesPerformance,"%")
        print("Average positive trades :", AveragePercentagePositivTrades, "%")
        print("Average negative trades :", AveragePercentageNegativTrades, "%")

        print("\n----- Trades Reasons -----")"""
        reasons = dico_dt[pairName]['reason'].unique()
        for r in reasons:
            pass
            # print(r+" number :", dico_dt[pairName].groupby('reason')['date'].nunique()[r])

        result_data_frame = result_data_frame.append({"pairName": pairName,
                                                      "trixLength": trixLength,
                                                      "trixSignal": trixSignal,
                                                      "ema200": ema200,
                                                      "stochTop": stochTop,
                                                      "stochBottom": stochBottom,
                                                      "stoch_rsi": stoch_rsi,
                                                      "Period start": str(dfTest.index[0]),
                                                      "Period end": str(dfTest.index[len(dfTest) - 1]),
                                                      "initalWallet": initalWallet,
                                                      "Final balance ($)": round(wallet, 2),
                                                      "Performance vs US Dollar (%)": round(algoPercentage, 2),
                                                      "Buy and Hold Performence(%) ": round(holdPercentage, 2),
                                                      "Performance vs Buy and Hold (%)": round(vsHoldPercentage, 2),
                                                      "Best trade (%)": bestTrade,
                                                      "Worst trade (%)": worstTrade,
                                                      "Worst drawBack(%)": str(100 * round(dt['drawBack'].min(), 2)),
                                                      "Total fees ($)": round(dt['frais'].sum(), 2),
                                                      "Total trades on period": totalTrades,
                                                      "Number of positive trades": totalGoodTrades,
                                                      "Number of negative trades": totalBadTrades,
                                                      "Trades win rate ratio (%)": round(winRateRatio, 2),
                                                      "Average trades performance (%)": tradesPerformance,
                                                      "Average positive trades (%)": AveragePercentagePositivTrades,
                                                      "Average negative trades (%)": AveragePercentageNegativTrades,
                                                      },
                                                     ignore_index=True)


list_thread = []
for trixSignal in trixSignal_plage:
    for trixLength in trixLength_plage:
        for stochTop in stochTop_plage:
            for stochBottom in stochBottom_plage:
                for stoch_rsi in stoch_rsi_plage:
                    if (len(list_thread) == 300):
                        for ii in list_thread:
                            ii.join()
                        list_thread = []

                    t = Thread(target=trainding_v2())
                    list_thread.append(t)
                    t.start()

if (len(list_thread) != 0):
    for i in list_thread:
        i.join()

end_time = perf_counter()
print(f'Alors le code optimisé a couté {end_time - start_time : 0.2f} secondes pour se completer.')


1 

9 21 0.8 0.1 12
Indicators loaded 100% for ETHUSDT
Indicators loaded 100% for BTCUSDT


/tmp/ipykernel_499506/3328130585.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dict_cryptos[pairName].drop(dict_cryptos[pairName].columns.difference(['open','high','low','close','volume']), 1, inplace=True)


2 

9 21 0.8 0.1 13
Indicators loaded 100% for ETHUSDT
Indicators loaded 100% for BTCUSDT


/tmp/ipykernel_499506/3328130585.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dict_cryptos[pairName].drop(dict_cryptos[pairName].columns.difference(['open','high','low','close','volume']), 1, inplace=True)


3 

9 21 0.8 0.1 14
Indicators loaded 100% for ETHUSDT
Indicators loaded 100% for BTCUSDT


...

3071 

11 24 0.94 0.24 14
Indicators loaded 100% for ETHUSDT
Indicators loaded 100% for BTCUSDT


/tmp/ipykernel_499506/3328130585.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dict_cryptos[pairName].drop(dict_cryptos[pairName].columns.difference(['open','high','low','close','volume']), 1, inplace=True)


3072 

11 24 0.94 0.24 15
Indicators loaded 100% for ETHUSDT
Indicators loaded 100% for BTCUSDT


/tmp/ipykernel_499506/3328130585.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dict_cryptos[pairName].drop(dict_cryptos[pairName].columns.difference(['open','high','low','close','volume']), 1, inplace=True)


Alors le code optimisé a couté  1282.92 secondes pour se completer.


In [20]:
result_data_frame = result_data_frame[['trixLength', 'trixSignal', 'ema200', 'stochTop','stochBottom', 'stoch_rsi',
                   'pairName','Period start','Period end','Final balance ($)','Buy and Hold Performence(%) ']]

result_data_frame = result_data_frame.groupby(by=['pairName'], 
                          axis=0, level=None, as_index=True, sort=True, group_keys=True, observed=False, dropna=True)

In [21]:
#enregistrement données 
path =r'C:\Users\aniss\Desktop\crpt\workspace_Crypto_robot\backtest_tools-main\backtest'
with pd.ExcelWriter(path+'/result_back_test.xlsx') as writer:
    for name, group in result_data_frame:
        group.to_excel(writer,sheet_name=f' {name}')
        


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\aniss\\Desktop\\crpt\\workspace_Crypto_robot\\backtest_tools-main\\backtest/result_back_test.xlsx'

## Traiding V1

In [ ]:
def algo_traiding(dict_df_indicated, result_data_frame, startDate, endDate, stochTop, stochBottom):
  dico_dt = {}   
  for pairName in  dict_df_indicated :  
    dfTest = None
    # dfTest = df.copy()
    
    if endDate == "" :
      endDate=None

    # -- Ifyou want to run your BackTest on a specific period, uncomment the line below --
    dfTest = dict_cryptos[pairName][startDate : endDate]

    # -- Definition of dt, that will be the dataset to do your trades analyses --
    dt = None
    dt = pd.DataFrame(columns = ['date','position', 'reason', 'price', 'frais' ,'fiat', 'coins', 'wallet', 'drawBack'])

    # -- You can change variables below --
    usdt = 1000
    makerFee = 0.0002
    takerFee = 0.0007

    # -- Do not touch these values --
    initalWallet = usdt
    wallet = usdt
    coin = 0
    lastAth = 0
    previousRow = dfTest.iloc[0]
    stopLoss = 0
    takeProfit = 500000
    buyReady = True
    sellReady = True

    # -- Condition to BUY market --
    def buyCondition(row, previousRow):
      if row['TRIX_HISTO'] > 0 and row['STOCH_RSI'] < stochTop:
        return True
      else:
        return False

    # -- Condition to SELL market --  
    def sellCondition(row, previousRow):
      if row['TRIX_HISTO'] < 0 and row['STOCH_RSI'] > stochBottom:
        return True
      else:
        return False

    # -- Iteration on all your price dataset (df) --
    for index, row in dfTest.iterrows():
      # -- Buy market order --
      if buyCondition(row, previousRow) and usdt > 0 and buyReady == True:
        # -- You can define here at what price you buy --
        buyPrice = row['close']

        # -- Define the price of you SL and TP or comment it if you don't want a SL or TP --
        # stopLoss = buyPrice - 0.02 * buyPrice
        # takeProfit = buyPrice + 0.04 * buyPrice

        coin = usdt / buyPrice
        fee = takerFee * coin
        coin = coin - fee
        usdt = 0
        wallet = coin * row['close']

        # -- Check if your wallet hit a new ATH to know the drawBack --
        if wallet > lastAth:
          lastAth = wallet

        # -- You can uncomment the line below if you want to see logs --
        # print("Buy COIN at",buyPrice,'$ the', index)

        # -- Add the trade to DT to analyse it later --
        myrow = {'date': index, 'position': "Buy", 'reason':'Buy Market Order','price': buyPrice,'frais': fee,'fiat': usdt,'coins': coin,'wallet': wallet,'drawBack':(wallet-lastAth)/lastAth}
        dt = dt.append(myrow,ignore_index=True)
      
      # -- Stop Loss --
      elif row['low'] < stopLoss and coin > 0:
        sellPrice = stopLoss
        usdt = coin * sellPrice
        fee = makerFee * usdt
        usdt = usdt - fee
        coin = 0
        buyReady = False
        wallet = usdt

        # -- Check if your wallet hit a new ATH to know the drawBack --
        if wallet > lastAth:
          lastAth = wallet
          
        # -- You can uncomment the line below if you want to see logs --
        # print("Sell COIN at Stop Loss",sellPrice,'$ the', index)

        # -- Add the trade to DT to analyse it later --
        myrow = {'date': index,'position': "Sell", 'reason':'Sell Stop Loss','price': sellPrice,'frais': fee,'fiat': usdt,'coins': coin,'wallet': wallet,'drawBack':(wallet-lastAth)/lastAth}
        dt = dt.append(myrow,ignore_index=True)    

      # -- Sell Market Order --
      elif sellCondition(row, previousRow) and coin > 0 and sellReady == True:

        # -- You can define here at what price you buy --
        sellPrice = row['close']
        usdt = coin * sellPrice
        fee = takerFee * usdt
        usdt = usdt - fee
        coin = 0
        buyReady = True
        wallet = usdt

        # -- Check if your wallet hit a new ATH to know the drawBack --
        if wallet > lastAth:
          lastAth = wallet

        # -- You can uncomment the line below if you want to see logs --  
        # print("Sell COIN at",sellPrice,'$ the', index)

        # -- Add the trade to DT to analyse it later --
        myrow = {'date': index,'position': "Sell", 'reason':'Sell Market Order','price': sellPrice,'frais': fee,'fiat': usdt,'coins': coin,'wallet': wallet,'drawBack':(wallet-lastAth)/lastAth}
        dt = dt.append(myrow,ignore_index=True)
      
      previousRow = row

    # -- BackTest Analyses --
    dt = dt.set_index(dt['date'])
    dt.index = pd.to_datetime(dt.index)
    dt['resultat'] = dt['wallet'].diff()
    dt['resultat%'] = dt['wallet'].pct_change()*100
    dt.loc[dt['position']=='Buy','resultat'] = None
    dt.loc[dt['position']=='Buy','resultat%'] = None

    dt['tradeIs'] = ''
    dt.loc[dt['resultat']>0,'tradeIs'] = 'Good'
    dt.loc[dt['resultat']<=0,'tradeIs'] = 'Bad'

    iniClose = dfTest.iloc[0]['close']
    lastClose = dfTest.iloc[len(dfTest)-1]['close']
    holdPercentage = ((lastClose - iniClose)/iniClose) * 100
    algoPercentage = ((wallet - initalWallet)/initalWallet) * 100
    vsHoldPercentage = ((algoPercentage - holdPercentage)/holdPercentage) * 100

    try:
        tradesPerformance = round(dt.loc[(dt['tradeIs'] == 'Good') | (dt['tradeIs'] == 'Bad'), 'resultat%'].sum()
                / dt.loc[(dt['tradeIs'] == 'Good') | (dt['tradeIs'] == 'Bad'), 'resultat%'].count(), 2)
    except:
        tradesPerformance = 0
        print("/!\ There is no Good or Bad Trades in your BackTest, maybe a problem...")

    try:
        totalGoodTrades = dt.groupby('tradeIs')['date'].nunique()['Good']
        AveragePercentagePositivTrades = round(dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].sum()
                                              / dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].count(), 2)
        idbest = dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].idxmax()
        bestTrade = str(
            round(dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].max(), 2))
    except:
        totalGoodTrades = 0
        AveragePercentagePositivTrades = 0
        idbest = ''
        bestTrade = 0
        print("/!\ There is no Good Trades in your BackTest, maybe a problem...")

    try:
        totalBadTrades = dt.groupby('tradeIs')['date'].nunique()['Bad']
        AveragePercentageNegativTrades = round(dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].sum()
                                              / dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].count(), 2)
        idworst = dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].idxmin()
        worstTrade = round(dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].min(), 2)
    except:
        totalBadTrades = 0
        AveragePercentageNegativTrades = 0
        idworst = ''
        worstTrade = 0
        print("/!\ There is no Bad Trades in your BackTest, maybe a problem...")

    totalTrades = totalBadTrades + totalGoodTrades
    #print(totalTrades)
    winRateRatio = (totalGoodTrades/totalTrades) * 100

    reasons = dt['reason'].unique()
    dico_dt[pairName] = dt
    """

    print("Pair Symbol :",pairName)
    print("Period : [" + str(dfTest.index[0]) + "] -> [" +
          str(dfTest.index[len(dfTest)-1]) + "]")
    print("Starting balance :", initalWallet, "$")

    print("\n----- General Informations -----")
    print("Final balance :", round(wallet, 2), "$")
    print("Performance vs US Dollar (%):", round(algoPercentage, 2), "%")
    print("Buy and Hold Performence :", round(holdPercentage, 2), "%")
    print("Performance vs Buy and Hold :", round(vsHoldPercentage, 2), "%")
    print("Best trade : ",bestTrade, "%, the", idbest)
    print("Worst trade :", worstTrade, "%, the", idworst)
    print("Worst drawBack :", str(100*round(dt['drawBack'].min(), 2)), "%")
    print("Total fees : ", round(dt['frais'].sum(), 2), "$")

    print("\n----- Trades Informations -----")
    print("Total trades on period :",totalTrades)
    print("Number of positive trades :", totalGoodTrades)
    print("Number of negative trades : ", totalBadTrades)
    print("Trades win rate ratio :", round(winRateRatio, 2), '%')
    print("Average trades performance :",tradesPerformance,"%")
    print("Average positive trades :", AveragePercentagePositivTrades, "%")
    print("Average negative trades :", AveragePercentageNegativTrades, "%")

    print("\n----- Trades Reasons -----")"""
    reasons = dico_dt[pairName]['reason'].unique()
    for r in reasons:
        pass
        #print(r+" number :", dico_dt[pairName].groupby('reason')['date'].nunique()[r])   

    result_data_frame = result_data_frame.append({"pairName" : pairName,
                                                  "trixLength": trixLength,
                                                  "trixSignal":trixSignal,
                                                  "ema200": ema200,
                                                  "stochTop":stochTop,
                                                  "stochBottom":stochBottom,
                                                  "stoch_rsi":stoch_rsi,
                                                  "Period start" :str(dfTest.index[0]), 
                                                  "Period end" :str(dfTest.index[len(dfTest)-1]),
                                                  "initalWallet" :initalWallet,
                                                  "Final balance ($)" :round(wallet, 2),
                                                  "Performance vs US Dollar (%)":round(algoPercentage, 2),
                                                  "Buy and Hold Performence(%) ": round(holdPercentage, 2),
                                                  "Performance vs Buy and Hold (%)": round(vsHoldPercentage, 2),
                                                  "Best trade (%)" : bestTrade,
                                                  "Worst trade (%)" : worstTrade, 
                                                  "Worst drawBack(%)" : str(100*round(dt['drawBack'].min(), 2)),
                                                  "Total fees ($)" :  round(dt['frais'].sum(), 2),
                                                  "Total trades on period" : totalTrades,
                                                  "Number of positive trades" : totalGoodTrades,
                                                  "Number of negative trades" :  totalBadTrades,
                                                  "Trades win rate ratio (%)" : round(winRateRatio, 2),
                                                  "Average trades performance (%)" : tradesPerformance,
                                                  "Average positive trades (%)" : AveragePercentagePositivTrades,
                                                  "Average negative trades (%)" : AveragePercentageNegativTrades,
                                                  },
                                                  ignore_index=True)       
  return result_data_frame

In [ ]:
stochTop_plage = np.arange(85,87,1)/100
stochBottom_plage = np.arange(25,26,1)/100
stoch_rsi_plage = np.arange(12,13,1)
trixLength_plage = np.arange(9,10,1)
trixSignal_plage = np.arange(22,24,1)
ema200 =200

l1 = stochTop_plage
l2 = stochBottom_plage
l3 = stoch_rsi_plage
for i,j,k in map(l1,l2,l3):
    print(i,j,k)

In [ ]:

from multiprocessing import Pool
def sum_up_to(number):
    return sum(range(1, number + 1))

a_pool = Pool()
result = a_pool.map(sum_up_to, range(10))
print(result)

[0, 1, 3, 6, 10, 15, 21, 28, 36, 45]
